In [1]:
import pandas as pd
import numpy as np
import time
import requests
import re

In [2]:
# Helper functions
def timer_function(func, data, n_run=1):
    ti = time.time()
    for i in range(n_run):
        func(data)
    tf = time.time()
    
    print(f'Runtime: {(tf - ti)/n_run} s')

def read_input_pd(file_path=None):
    df_data = pd.read_csv(file_path, sep=',', header=None)
    
    return df_data

def read_input_lst(file_path=None):
    with open(file_path,'r') as f:
        lines = f.read().splitlines()

    input_arr = list(lines)


    return input_arr

def scrape_data(day):
    """
    fetches the input data from the website
    day: the day of december to fetch
    """
    # Fetch session cookie value from local file
    with open("session.txt",'r') as f:
        key = f.read()
    cookie = {'session': key}
    url= f"https://adventofcode.com/2021/day/{day}/input"

    with requests.Session() as s:
        r = s.get(url, cookies = cookie).text
        # Data always end with a blanc line. Data is split on line change
        input_lst = re.split('\n', r)[:-1]

    return input_lst

# December 1


In [3]:
# data import
input_test = np.array([199,200,208,210,200,207,240,269,260,263])
input_1 = scrape_data(day=1)
input_1 = [int(i) for i in input_1]
print('Number of input rows:', len(input_1))

Number of input rows: 2000


In [4]:
# Part 1
def dec1_part1(data):
    answer = np.sum(np.diff(data) > 0)
    return answer

answer = dec1_part1(input_1)
print(f'Solution december 1. - part 1: ', answer)
timer_function(func=dec1_part1, data=input_1, n_run=1000)

Solution december 1. - part 1:  1583
Runtime: 0.0001440882682800293 s


In [5]:
# Part 2
def dec1_part2(data):
    window = 3
    rolling_sum =[np.sum(data[i:i + window]) for i in range(0, len(data) + 1 - window)]
    answer = dec1_part1(rolling_sum)

    return rolling_sum, answer

rolling_sum, answer = dec1_part2(input_1)
print(f'Solution december 1. - part 2: ', answer)
timer_function(func=dec1_part2,data=input_1, n_run=10)

Solution december 1. - part 2:  1627
Runtime: 0.008801865577697753 s


# December 2


In [6]:
input_test = ['forward 5','down 5','forward 8','up 3','down 8','forward 2']
input_2 = scrape_data(day=2)
print('Number of input rows:', len(input_2))

Number of input rows: 1000


In [7]:
# Solution and test input
def split_direction(df, direction, sign):
    mask = df['raw'].str.contains(direction)
    df.loc[mask, 'direction'] = direction[0]
    df.loc[mask, 'step'] = df.loc[mask, 'raw'].str.replace(direction, '').astype(int)*sign
    
    return df

def dec2_part1(data):
    df_data = pd.DataFrame(data).rename(columns={0:'raw'})
    
    df_data = split_direction(df_data, 'forward ', +1)
    df_data = split_direction(df_data, 'down ', +1)
    df_data = split_direction(df_data, 'up ', -1)
    
    mask_fwd = df_data['direction'] == 'f'
    pos_horizontal = df_data.loc[mask_fwd, 'step'].sum()
    pos_vertical = df_data.loc[~mask_fwd, 'step'].sum()
    
    answer = pos_horizontal*pos_vertical
    
    return pos_horizontal, pos_vertical, df_data, answer


pos_horizontal, pos_vertical, df_input, answer = dec2_part1(input_2)
print(f'Position (horizontal/vertical):', pos_horizontal, ',',pos_vertical)
print(f'Solution: ', answer)
timer_function(func=dec2_part1,data=input_2, n_run=10)

Position (horizontal/vertical): 1939.0 , 1109.0
Solution:  2150351.0
Runtime: 0.007203125953674316 s


In [8]:
# December  2 - Part 2
def dec2_part2(df):    
    mask_fwd = df['direction'] == 'f'
    
    df['aim_step'] = 0
    df.loc[~mask_fwd, 'aim_step'] = df.loc[~mask_fwd, 'step']
    df['aim'] = df['aim_step'].cumsum()

    pos_horizontal = df.loc[mask_fwd, 'step'].sum()
    pos_vertical = (df.loc[mask_fwd, 'step']*df.loc[mask_fwd, 'aim']).sum()
    
    answer = pos_horizontal*pos_vertical
    
    return pos_horizontal, pos_vertical, df, answer

pos_horizontal, pos_vertical, df_input, answer = dec2_part2(df_input)
print(f'Position (horizontal/vertical):', pos_horizontal, ',',pos_vertical)
print(f'Solution: ', answer)
timer_function(func=dec2_part2,data=df_input, n_run=10)

Position (horizontal/vertical): 1939.0 , 950357.0
Solution:  1842742223.0
Runtime: 0.0018006563186645508 s


# December 3

In [9]:
input_test = ['00100','11110','10110','10111','10101','01111','00111','11100','10000','11001','00010','01010']
input_3 = scrape_data(day=3)
print('Number of input rows:', len(input_3))

Number of input rows: 1000


In [10]:
# Solution part 1 and test
def dec3_part1(data):
    n_digit = len(data[0])
    n_row = len(data)
    
    digit_sum = np.zeros(n_digit)
    
    for row in data:
        row_lst = [int(i) for i in row]
        digit_sum = np.add(digit_sum, row_lst)
    
    mask = digit_sum >= n_row/2
    gamma = ''.join(map(str, 1*(mask)))
    epsilon = ''.join(map(str, 1*(~mask)))
    
    answer = int(gamma,2)*int(epsilon,2)
    
    return gamma, epsilon, answer
    
print(dec3_part1(input_3))
timer_function(func=dec3_part1,data=input_3, n_run=10)

('010001110111', '101110001000', 3374136)
Runtime: 0.004100656509399414 s


In [11]:
def find_rating(rating, bit):
    n_digit = len(rating[0])
    n_row = len(rating)
    idx = 0
    
    while (len(rating) > 1) & (idx < n_digit):
        half_len = len(rating)/2
        digit_sum = 0
        for row in rating:
            digit_sum += int(row[idx])
            
        digit_even = digit_sum == half_len
        
        if digit_even:
            rating = [entry for entry in rating if entry[idx] == bit]
        else:
            if bit == '0':
                digit_match = 1*(digit_sum < half_len)
            else:
                digit_match = 1*(digit_sum > half_len)

            rating = [entry for entry in rating if entry[idx] == str(digit_match)]

        idx += 1
    
    # select the only remaining correct entry if more exist
    rating = [entry for entry in rating if entry[-1] == bit][0]
    
    return rating

def dec3_part2(data):
    o_rating = find_rating(rating=data, bit='1')
    co2_rating = find_rating(rating=data, bit='0')
        
    answer = int(o_rating,2)*int(co2_rating,2)
    
    return o_rating, co2_rating, answer

print(dec3_part2(input_3))
timer_function(func=dec3_part2,data=input_3, n_run=10)

('011101110101', '100100010010', 4432698)
Runtime: 0.001200103759765625 s


# December 4

In [12]:
input_test = [
    '7,4,9,5,11,17,23,2,0,14,21,24,10,16,13,6,15,25,12,22,18,20,8,19,3,26,1',
    '',
    '22 13 17 11  0',
    '8  2 23  4 24',
    '21  9 14 16  7',
    '6 10  3 18  5',
     '1 12 20 15 19',
    '',
    '3 15  0  2 22',
    '9 18 13 17  5',
    '19  8  7 25 23',
    '20 11 10 24  4',
    '14 21 16 12  6',
    '',
    '14 21 17 24  4',
    '10 16 15  9 19',
    '18  8 23 26 20',
    '22 11 13  6  5',
     '2  0 12  3  7']
input_4 = scrape_data(day=4)

In [13]:
def create_data_structure(data):
    numbers = list(map(int, re.split(',', data[0])))
    idx_start = 2
    idx_end = len(data) - 1
    n_size = 5

    boards = []
    for board_start in range(idx_start, idx_end, n_size + 1):
        board = []
        for idx in range(board_start, board_start + n_size):
            board.append(list(map(int, re.split('\s+', data[idx].strip()))))


        for col in np.transpose(board):
            board.append(col)
            
        boards.append(board)

    board_sets = [[set(row) for row in board] for board in boards]
    return numbers, board_sets

numbers, board_sets = create_data_structure(input_4)

In [14]:
def dec4_part1(data):
    numbers, board_sets = create_data_structure(data)
    
    keep_going = True
    for idx, n in enumerate(numbers):
        for n_board, board in enumerate(board_sets):
            for row in board:
                row.discard(n)

                if len(row) == 0:
                    keep_going = False
                    board_winner = board

        if keep_going:
            continue
        else:
            break
            
            
    remainders = set(i for row in board_winner for i in row)
    winner_board = idx
    answer = sum(remainders)*n
    return n_board, answer

print('Answer:', dec4_part1(input_4))
timer_function(func=dec4_part1,data=input_4, n_run=10)

Answer: (99, 46920)
Runtime: 0.004499554634094238 s


In [15]:
def dec4_part2(data):
    numbers, board_sets = create_data_structure(data)
    keep_going = True
    for idx, n in enumerate(numbers):
        del_board = []
        for n_board, board in enumerate(board_sets):

            for row in board:
                row.discard(n)
                if len(row) == 0:
                    del_board.append(n_board)
                    last_board = board
        
        if len(del_board)>0:
            board_tmp = board_sets.copy()
            
            for board in set(del_board):
                board_sets.remove(board_tmp[board])
            
            if len(board_sets) == 0:
                keep_going = False

        if keep_going:
            continue
        else:
            break
            
            
    remainders = set(i for row in last_board for i in row)
    answer = sum(remainders)*n
    return answer


print('Answer:', dec4_part2(input_4))
timer_function(func=dec4_part2,data=input_4, n_run=10)

Answer: 12635
Runtime: 0.008201074600219727 s


# December 5

In [16]:
input_test = [
    '0,9 -> 5,9',
    '8,0 -> 0,8',
    '9,4 -> 3,4',
    '2,2 -> 2,1',
    '7,0 -> 7,4',
    '6,4 -> 2,0',
    '0,9 -> 2,9',
    '3,4 -> 1,4',
    '0,0 -> 8,8',
    '5,5 -> 8,2']
input_5 = scrape_data(day=5)

In [17]:
# Part 1
def data_prep(data):
    data_clean = [tuple(map(int, row.replace(' -> ', ',').split(','))) for row in data]
    
    n_size = np.max(data_clean) + 1
    
    grid = np.zeros((n_size,n_size))
    
    return data_clean, grid

def add_line(coords):
    if coords[0] == coords[2]:
        x = coords[0]
        y1 = min(coords[1], coords[3])
        y2 = max(coords[1], coords[3])

        for yi in range(y1,y2+1):
            grid[yi][x] += 1
    else:
        y = coords[1]
        x1 = min(coords[0], coords[2])
        x2 = max(coords[0], coords[2])

        for xi in range(x1,x2+1):
            grid[y][xi] += 1

def dec5_part1(data):
    for coord in line_coords:
        add_line(coords=coord)
    return np.sum(grid > 1)


line_coords, grid = data_prep(input_5)
# ONly horizontal/vertical
line_coords = [row for row in line_coords if (row[0]==row[2]) | (row[1]==row[3])]

print('Answer:', dec5_part1(line_coords))
timer_function(func=dec5_part1,data=input_5, n_run=10)

Answer: 8060
Runtime: 0.05291273593902588 s


In [18]:
# Part 2
def add_line_diag(coords):
    x1, y1, x2, y2 = coords
    x_step = np.sign(x2 - x1)
    y_step = np.sign(y2 - y1)
    

    if x1 == x2:
        for yi in range(y1, y2 + y_step, y_step):
            grid[yi][x1] += 1
    elif y1 == y2:
        for xi in range(x1, x2 + x_step, x_step):
            grid[y1][xi] += 1
    else:
        for xi,yi in zip(range(x1, x2 + x_step, x_step), range(y1, y2 + y_step, y_step)):
            grid[yi][xi] += 1
            

def dec5_part2(data):
    for coord in line_coords:
        add_line_diag(coords=coord)
    return np.sum(grid > 1)

line_coords, grid = data_prep(input_5)
print('Answer:', dec5_part2(line_coords))
timer_function(func=dec5_part2,data=input_5, n_run=10)

Answer: 21577
Runtime: 0.09313132762908935 s


# December 6

In [19]:
input_test = list(map(int, ['3,4,3,1,2'][0].split(',')))
input_6 = list(map(int, scrape_data(day=6)[0].split(',')))
input_test

[3, 4, 3, 1, 2]

In [20]:
# Part 1
def dec6_part1(fish_pop_initial):
    days = 80
    fish_pop = np.array(fish_pop_initial.copy(), dtype=int)
    for day in range(1, days + 1):
        mask_reset = fish_pop == 0

        fish_pop -= 1*(fish_pop >= 0)

        fish_pop[mask_reset] = 6
        fish_pop = np.append(fish_pop, np.sum(mask_reset, dtype=int)*[8])

    return fish_pop.astype(int)

print('Answer:', len(dec6_part1(input_6)))
timer_function(func=dec6_part1,data=input_6,n_run=10)

Answer: 362346
Runtime: 0.04811081886291504 s


In [21]:
# Part 2
def dec6_part2(fish_pop_initial):
    days = 256
    # Initial states of fish
    fish_states = np.zeros([9])
    for fish in fish_pop_initial:
        fish_states[fish] += 1
    
    # fish evolving logic
    evolve_map = [
        np.array([-1, 0, 0, 0, 0, 0, 1, 0, 1]),
        np.array([1, -1, 0, 0, 0, 0, 0, 0, 0]),
        np.array([0, 1, -1, 0, 0, 0, 0, 0, 0]),
        np.array([0, 0, 1, -1, 0, 0, 0, 0, 0]),
        np.array([0, 0, 0, 1, -1, 0, 0, 0, 0]),
        np.array([0, 0, 0, 0, 1, -1, 0, 0, 0]),
        np.array([0, 0, 0, 0, 0, 1, -1, 0, 0]),
        np.array([0, 0, 0, 0, 0, 0, 1, -1, 0]),
        np.array([0, 0, 0, 0, 0, 0, 0, 1, -1])]
    
    # fish population evolution
    for day in range(1, days + 1):
        for state, count in enumerate(fish_states):
            fish_states = fish_states + count*evolve_map[state]
        
    return np.sum(fish_states)

print('Answer:', dec6_part2(input_6))
timer_function(func=dec6_part2,data=input_6,n_run=10)

Answer: 1639643057051.0
Runtime: 0.005609703063964844 s


# December 7

In [22]:
input_test = list(map(int,['16,1,2,0,4,2,7,1,2,14'][0].split(',')))
input_7 = list(map(int,scrape_data(7)[0].split(',')))
print('Input length:', len(input_7))
print('Test input:', input_test)

Input length: 1000
Test input: [16, 1, 2, 0, 4, 2, 7, 1, 2, 14]


In [23]:
def dec7_part1(data):
    p = np.array(data)
    pos_min = int(np.median(p))
    fuel_min = np.sum(np.abs(p - pos_min))
    
    return pos_min, fuel_min

min_pos, min_fuel = dec7_part1(input_7)
print(f'Optimal position {min_pos} cost a fuel total of {min_fuel}')
timer_function(func=dec7_part1,data=input_7,n_run=10)

Optimal position 331 cost a fuel total of 349769
Runtime: 0.00015053749084472656 s


In [24]:
def consecutive_sum(n):
    answer = n*(n + 1)/2
    return answer

def dec7_part2(data):
    p = np.array(data)
    pos = range(np.min(data), np.max(data) + 1)
    dp = [consecutive_sum(np.abs(p - i)) for i in pos]
    fuel = np.sum(dp, axis=1)
    
    return pos[np.argmin(fuel)], np.min(fuel)

min_pos, min_fuel = dec7_part2(input_7)
print(f'Optimal position at {min_pos} cost a fuel total of {min_fuel}')
timer_function(func=dec7_part2,data=input_7,n_run=10)

Optimal position at 479 cost a fuel total of 99540554.0
Runtime: 0.028005456924438475 s


In [25]:
input_test = [
    'be cfbegad cbdgef fgaecd cgeb fdcge agebfd fecdb fabcd edb |fdgacbe cefdb cefbgd gcbe',
    'edbfga begcd cbg gc gcadebf fbgde acbgfd abcde gfcbed gfec |fcgedb cgb dgebacf gc',
    'fgaebd cg bdaec gdafb agbcfd gdcbef bgcad gfac gcb cdgabef |cg cg fdcagb cbg',
    'fbegcd cbd adcefb dageb afcb bc aefdc ecdab fgdeca fcdbega |efabcd cedba gadfec cb',
    'aecbfdg fbg gf bafeg dbefa fcge gcbea fcaegb dgceab fcbdga |gecf egdcabf bgf bfgea',
    'fgeab ca afcebg bdacfeg cfaedg gcfdb baec bfadeg bafgc acf |gebdcfa ecba ca fadegcb',
    'dbcfg fgd bdegcaf fgec aegbdf ecdfab fbedc dacgb gdcebf gf |cefg dcbef fcge gbcadfe',
    'bdfegc cbegaf gecbf dfcage bdacg ed bedf ced adcbefg gebcd |ed bcgafe cdgba cbgef',
    'egadfb cdbfeg cegd fecab cgb gbdefca cg fgcdab egfdb bfceg |gbdfcae bgc cg cgb',
    'gcafb gcf dcaebfg ecagb gf abcdeg gaef cafbge fdbac fegbdc |fgae cfgab fg bagce']
input_test = [i.split(' |') for i in input_test]
input_8=scrape_data(8)
input_8 = [i.split(' | ') for i in input_8]
print('Input length:', len(input_7))
print('Test input:', input_test)

Input length: 1000
Test input: [['be cfbegad cbdgef fgaecd cgeb fdcge agebfd fecdb fabcd edb', 'fdgacbe cefdb cefbgd gcbe'], ['edbfga begcd cbg gc gcadebf fbgde acbgfd abcde gfcbed gfec', 'fcgedb cgb dgebacf gc'], ['fgaebd cg bdaec gdafb agbcfd gdcbef bgcad gfac gcb cdgabef', 'cg cg fdcagb cbg'], ['fbegcd cbd adcefb dageb afcb bc aefdc ecdab fgdeca fcdbega', 'efabcd cedba gadfec cb'], ['aecbfdg fbg gf bafeg dbefa fcge gcbea fcaegb dgceab fcbdga', 'gecf egdcabf bgf bfgea'], ['fgeab ca afcebg bdacfeg cfaedg gcfdb baec bfadeg bafgc acf', 'gebdcfa ecba ca fadegcb'], ['dbcfg fgd bdegcaf fgec aegbdf ecdfab fbedc dacgb gdcebf gf', 'cefg dcbef fcge gbcadfe'], ['bdfegc cbegaf gecbf dfcage bdacg ed bedf ced adcbefg gebcd', 'ed bcgafe cdgba cbgef'], ['egadfb cdbfeg cegd fecab cgb gbdefca cg fgcdab egfdb bfceg', 'gbdfcae bgc cg cgb'], ['gcafb gcf dcaebfg ecagb gf abcdeg gaef cafbge fdbac fegbdc', 'fgae cfgab fg bagce']]


In [27]:
digit_dct = {
    0: ('a','b','c','e','f','g'),
    1: ('c','f'),
    2: ('a','c','d','e','g'),
    3: ('a','c','d','f','g'),
    4: ('b','c','d','f'),
    5: ('a','b','d','f','g'),
    6: ('a','b','d','e','f','g'),
    7: ('a','c','f'),
    8: ('a','b','c','d','e','f','g'),
    9: ('a','b','c','d','f','g')}
digit_cnt = dict([(i,len(digit_dct[i])) for i in range(9+1)])

unique_digit_cnt = [1,4,7,8]
unique_len = set([digit_cnt[d] for d in unique_digit_cnt])

def dec8_part1(data):
    value_cnt = 0
    for segment in data:
        value_cnt += np.sum([(len(o) in unique_len) for o in segment[1].split(' ')])
    
    return value_cnt

print('Solution part 1:', dec8_part1(input_8))
timer_function(func=dec8_part1,data=input_8,n_run=10)

Solution part 1: 318
Runtime: 0.0011033058166503907 s


In [ ]:
# 1: 2:4, 3:3, 5:4, 0:4, 6:5, 9:4
# 7: 2:3, 3:2, 5:3, 0:3, 6:4, 9:3
# 4: 2:3, 3:2, 5:2, 0:3, 6:3, 9:2
        
def dec8_part2(data):
    digit_sum = 0
    for line in data:
        segment = [list(d) for d in line[0].split(' ')]
        output = [set(list(d)) for d in line[1].split(' ')]
        digits = sorted([set(d) for d in segment], key=len)

        # known codes
        digit_code = {1: digits[0], 7: digits[1], 4: digits[2], 8: digits[-1]}
        
        # codes with length 5: (2,3,5)
        for d in range(3,6):
            d1 = len(digits[d] - digit_code[1])
            d4 = len(digits[d] - digit_code[4])
            if d1 == 3:
                digit_code[3] = digits[d]
            elif d4 == 3:
                digit_code[2] = digits[d]
            else:
                digit_code[5] = digits[d]
                
        # codes with length 6: (0,6,9)
        for d in range(6,9):
            d1 = len(digits[d] - digit_code[1])
            d4 = len(digits[d] - digit_code[4])
            if d1 == 5:
                digit_code[6] = digits[d]
            elif d4 == 2:
                digit_code[9] = digits[d]
            else:
                digit_code[0] = digits[d]
                
        digit_decifer = [digit_code[i] for i in range(9+1)] # letter-codes ordered after numerical value
        digit = int(''.join([str(digit_decifer.index(o)) for o in output]))
        digit_sum += digit

    return digit_sum

print('Solution part 2:', dec8_part2(input_8))
timer_function(func=dec8_part2,data=input_8,n_run=10)